In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import seaborn as sns





In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
#train.head()
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print (train.shape)
print (test.shape)

(42000, 785)
(28000, 784)


In [4]:
# Just data preparation, data comes with a label column that defines the outcome digit number

train_y = train['label'].astype('float32')
train_x = train.drop(['label'], axis=1).astype('float32')
test_x = test.astype('float32')
train_x.shape, train_y.shape, test_x.shape


((42000, 784), (42000,), (28000, 784))

In [5]:
# More data preparation, data comes as 784 pixels in a row instead of 28x28

train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x / 255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x / 255.0
train_x.shape, test_x.shape 

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [6]:
# One Hot Encoding

train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [7]:
print(train['label'].head()) # same as print(train_y.head())

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


In [8]:
print(train_y[0:5, :]) 

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)), #32 Anzahl der Filters (Features, Patches) als Output ; (3,3) ist nicht stride sondern die kernel_size ; #Tipp: Use (1,1) Kernel_size when adjusting the number of layers .. we'll keep that in mind
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), # (2,2) is just the pool size
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides = (2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'), #just a regular neural network layer - for this I need to study regular neural networks
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(10, activation='softmax') 
])
model.summary() 


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                    

In [10]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.99):
            print('\nReached 99.9% accuracy so cancel training')
            self.model.stop_training = True

callback = myCallback()


In [13]:
Optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Mit Compile wird das Model optimiert, also der Fehler minimiert (in dem Fall Cross Entropy, für Regression ist es der MSE - Mean squared error)

model.fit(train_x, train_y, batch_size = 50, epochs = 20, callbacks = [callback]) 

# Mit Fit wird das Model trainiert. Batch_size teilt die 42000 Images in Portionen for easier computing performance. 
# Epochs Durchgänge um Fehler zu reduzieren. Callback stoppt das Training wenn Fehlergenauigkeit erreicht ist.
# Fit only stops if 0.999 is reached. If not it performs all 20 epochs


Epoch 1/20


2022-11-08 08:55:30.146585: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


840/840 [==============================] - 19s 23ms/step - loss: 0.4251 - accuracy: 0.8589
Epoch 2/20
840/840 [==============================] - 19s 23ms/step - loss: 0.1069 - accuracy: 0.9689
Epoch 3/20
840/840 [==============================] - 19s 22ms/step - loss: 0.0797 - accuracy: 0.9763
Epoch 4/20
840/840 [==============================] - 19s 22ms/step - loss: 0.0663 - accuracy: 0.9799
Epoch 5/20
840/840 [==============================] - 19s 22ms/step - loss: 0.0563 - accuracy: 0.9830
Epoch 6/20
840/840 [==============================] - 19s 23ms/step - loss: 0.0472 - accuracy: 0.9857
Epoch 7/20
840/840 [==============================] - 18s 22ms/step - loss: 0.0442 - accuracy: 0.9868
Epoch 8/20
840/840 [==============================] - 18s 22ms/step - loss: 0.0395 - accuracy: 0.9882
Epoch 9/20
840/840 [==============================] - 19s 22ms/step - loss: 0.0364 - accuracy: 0.9894
Epoch 10/20
840/840 [==============================] - 19s 22ms/step - loss: 0.0331 - accurac

In [23]:
results = model.predict(test_x)
results = np.argmax(results, axis = 1)
results = pd.Series(results, name = 'Label')


875/875 [==============================] - 4s 5ms/step


In [25]:
print(results)

0        2
1        0
2        9
3        9
4        3
        ..
27995    9
27996    7
27997    3
27998    9
27999    2
Name: Label, Length: 28000, dtype: int64


In [27]:
submission = pd.concat([pd.Series(range(1,28001), name = 'ImageId'),results], axis=1)
#fügt eine Zeile hinzu die Predictions von 1 bis 28000 durchzählt

submission.to_csv('submission.csv', index=False)
#speichert meine Predictions als Table

model.save('model') #Note that model.save() is an alias for tf.keras.models.save_model()

#Frage: Warum kriege ich die Predictions nicht in Form des Hot Encoding raus? Ok das sind Kleinigkeiten

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [17]:
print(submission)

       ImageId  Label
0            1      2
1            2      0
2            3      9
3            4      9
4            5      3
...        ...    ...
27995    27996      9
27996    27997      7
27997    27998      3
27998    27999      9
27999    28000      2

[28000 rows x 2 columns]
